In [ ]:
# Improvements:
# Replace msising values with median or average and then see the impact on the metrics.
# Adding categerocial data

In [ ]:
# Group 3

In [ ]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
# load dataset
df = pd.read_csv('/Users/muhammadraza/Documents/GitHub/BIPM/Data Science/data/Dataset_Melbourne.csv', index_col=None)

In [ ]:
df.info()

In [ ]:
# Some features are missing for a lot of the data.
# Most are floats and some are objects

In [ ]:
from sklearn import set_config
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

## Instantiating the Linear Regression

lm = LinearRegression()
set_config(transform_output="pandas")

X = df[['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']]
y = df['Price']

imputer_median = SimpleImputer(strategy='median', add_indicator=True)
imputer_median.fit(np.array(df['Price']).reshape(-1, 1))  # Impute missing values in y
df[['Price', 'Imputer Price']] = imputer_median.transform(np.array(df['Price']).reshape(-1, 1))

y = df['Price']


# Stratify was taken out since its not relevant to regression.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

ct = ColumnTransformer(
    transformers=[
    ('imputer_x', imputer_median, ['Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Distance'])
    ],
    remainder='passthrough'
)

pipe = Pipeline([
    ('preprocessor', ct),
    ('classifier', lm)]
)

pipe.fit(X_train, y_train)

predictions = pipe.predict(X_test)

# EVALUATION METRICS
# ==================

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print("MAE:", mean_absolute_error(y_test, predictions))
print("RMSE:", mean_squared_error(y_test, predictions, squared=False))